#  <span style="color:orange">Natural Language Processing Tutorial (NLP102) - Level Intermediate</span>

**사용: PyCaret 2.0** <br />
**업데이트 날짜: August 24, 2020**

# 1.0 튜토리얼의 목적
자연어 처리 튜토리얼 (NLP102)에 오신 걸 환영합니다. 이 튜토리얼은 __[자연어 처리 튜토리얼 (NLP101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Natural%20Language%20Processing%20Tutorial%20Level%20Beginner%20-%20NLP101.ipynb)__ 을 완료한 사람을 위한 것 입니다. 이 튜토리얼에서 다룰 몇 가지 주요 개념은 초급 튜토리얼과 연결되어 있으므로 초급 튜토리얼을 보지 않았다면 다시 돌아가서 초급 튜토리얼을 진행하는 것이 좋습니다. 

이전의 튜토리얼에 기초하여, 이번 튜토리얼에서 다음 사항을 배울 것 입니다: 

* **사용자 지정 불용어:**  사용자 지정 불용어를 정하는 방법
* **토픽 모델 평가:**  토픽 모델의 성능를 평가하는 방법
* **하이퍼파라미터 튜닝:**  토픽 모델에 대한 하이퍼파라미터(# of topics)를 튜닝하는 방법
* **실험 기록:**  MLFlow 백엔드를 사용하여 Pycaret에서 실험을 기록하는 방법

소요 시간 : 대략 30분


# 1.1 PyCaret 설치
PyCaret를 설치하지 않았다면. 설치 방법은 __[초보자 튜토리얼](https://github.com/pycaret/pycaret/blob/master/tutorials/Natural%20Language%20Processing%20Tutorial%20Level%20Beginner%20-%20NLP101.ipynb)__ 을 참조하십시오.


# 1.2 사전 요구 사항
- Python 3.6 or greater
- PyCaret 2.0 or greater
- pycaret 저장소에서 데이터를 로드하기 위한 인터넷 연결
- 자연어 처리 튜토리얼 (NLP101) - Level Beginner 이수
- 자연 이항 분류 (CLF101) - Level Beginner 이수

# 1.3 Google colab 이용자:
Google colab에서 이 노트북을 실행하는 경우, 노트북 상단에서 아래의 셀 코드를 실행해야 대화형 시각 자료를 표시할 수 있습니다.<br/>
<br/>
`from pycaret.utils import enable_colab` <br/>
`enable_colab()`

# 1.4 참고:
- __[자연어 처리 튜토리얼 (NLP101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Natural%20Language%20Processing%20Tutorial%20Level%20Beginner%20-%20NLP101.ipynb)__
- __[자연어 처리 튜토리얼 (NLP103) - Level Expert](https://github.com/pycaret/pycaret/blob/master/tutorials/Natural%20Language%20Processing%20Tutorial%20Level%20Expert%20-%20NLP103.ipynb)__

# 2.0 튜토리얼 개요

이전 튜토리얼인 __[자연어 처리 튜토리얼 (NLP101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Natural%20Language%20Processing%20Tutorial%20Level%20Beginner%20-%20NLP101.ipynb)__ 에 기초하여 `custom_stopwords`를 전달한 후 본 튜토리얼에서 토픽 모델을 만들 것 입니다. 그런 다음 토픽 모델의 결과를 평가하고 이전 튜토리얼에서 만든 모델과 비교할 것입니다. 그리고 간섭값을 사용하여 토픽모델을 평가하는 방법과 감독 학습 대상을 최적화할 수 있는 PyCaret의 고유한 `tune_model()` 기능을 사용하는 방법을 알아볼 것입니다. 마지막으로 `pycaret.nlp` 모듈에서 추출한 토픽 정보를 사용하여 대출 부실을 예측할 수 있는 최적의 classifier을 찾기 위해 `pycaret.classifcation` 모듈의 토픽 모델 출력을 사용하여 튜토리얼을 마칠 것입니다.

시작합시다!

# 3.0 튜토리얼 Dataset

이 튜토리얼에서는 __[자연어 처리 튜토리얼 (NLP101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Natural%20Language%20Processing%20Tutorial%20Level%20Beginner%20-%20NLP101.ipynb)__ 에서 사용된 dataset을 사용 할 것입니다.. 우리의 깃허브 저장소에서 다운받을 수 있습니다 __[(다운하기)](https://github.com/pycaret/pycaret/blob/master/datasets/kiva.csv)__ 또는 PyCaret의 `get_data()` 기능을 사용해서 dataset을 가져올 수 있습니다 (인터넷 연결 필요)

# Dataset 승인:
Kiva Microfunds https://www.kiva.org/ 

# 4.0 데이터 가져오기

In [1]:
from pycaret.datasets import get_data
data = get_data('kiva')

,country,en,gender,loan_amount,nonpayment,sector,status
0,Dominican Republic,"""Banco Esperanza"" is a group of 10 women looki...",F,1225,partner,Retail,0
1,Dominican Republic,"""Caminemos Hacia Adelante"" or ""Walking Forward...",F,1975,lender,Clothing,0
2,Dominican Republic,"""Creciendo Por La Union"" is a group of 10 peop...",F,2175,partner,Clothing,0
3,Dominican Republic,"""Cristo Vive"" (""Christ lives"" is a group of 10...",F,1425,partner,Clothing,0
4,Dominican Republic,"""Cristo Vive"" is a large group of 35 people, 2...",F,4025,partner,Food,0


In [2]:
#check the shape of data
data.shape

(6818, 7)

In [3]:
# sampling the data to select only 1000 documents
data = data.sample(1000, random_state=786).reset_index(drop=True)
data.shape

(1000, 7)

# 5.0 PyCaret 환경 설정

`setup()` 기능은 PyCaret에서 환경을 초기화하고 NLP 문제 해결에 필수적인 몇 가지 텍스트 전처리 단계를 수행합니다. 이전 튜토리얼에서는 사용자 지정 불용어를 전달하지 않았으며, 이 튜토리얼에서는 `custom_stopwords` 파라미터를 사용하여 수행할 것입니다. 아래에서 전달된 모든 사용자 지정 불용어는 __[자연어 처리 튜토리얼 (NLP101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Natural%20Language%20Processing%20Tutorial%20Level%20Beginner%20-%20NLP101.ipynb)__ 에서 수행한 분석을 통해 얻어진 것입니다.(섹션 9.1을 참고하십시오). 이 단어들은 문서에서 빈도가 매우 높은 단어들입니다. 그만큼 정보보다 잡음을 더 더하고 있습니다. 사용자 지정 불용어 목록을 결정하는 것은 주관적인 결정이며 대부분 dataset에 대한 이해에서 유래합니다. 예를들어, 고객 대출 dataset을 사용하기 때문에 이 dataset에서는 '대출', '소득', '비즈니스', 'usd' 등과 같은 단어가 매우 명확합니다. 아래 `setup()` 에 전달된 나머지 파라미터는 이전 튜토리얼과 동일합니다.

In [4]:
from pycaret.nlp import *

In [5]:
exp_nlp102 = setup(data = data, target = 'en', session_id = 123,
                   custom_stopwords = ['loan', 'income', 'usd', 'many', 'also', 'make', 'business', 'buy', 
                                       'sell', 'purchase','year', 'people', 'able', 'enable', 'old', 'woman',
                                       'child', 'school'],
                   log_experiment = True, experiment_name = 'kiva1')

Description,Value
session_id,123
Documents,1000
Vocab Size,4552
Custom Stopwords,True


# 6.0 토픽 모델 생성

In [6]:
lda = create_model('lda')

In [7]:
plot_model(lda, plot = 'topic_distribution')

위의 출력을 __[이전 튜토리얼](https://github.com/pycaret/pycaret/blob/master/tutorials/Natural%20Language%20Processing%20Tutorial%20Level%20Beginner%20-%20NLP101.ipynb)__ 의 섹션 9.4와 비교할 경우 주제의 분포가 바뀌었다는 것을 알게 될 것입니다, 하지만 더 중요한 것은 bar 위에 마우스를 올려 놓을 때, 사용자 지정 금지어를 제거함으로써 약간의 잡음을 제거했기 때문에 키워드는 이 실험의 주제에 대한 더 나은 이해를 제공한다는 것입니다. 예를들어 `Topic 3`은 '머리카락', '살롱', '나무', '기계' 등의 키워드가 들어있어 무역 대출을 받으려는 고객들의 이야기로 보입니다.  `Topic 0`은 농업대출, `Topic 1`은 소매대출, 그리고 `Topic 2`은 국내대출에 관한 것으로 보입니다.

토픽 모델링은 매우 반복적인 기계 학습 작업이고, 올바른 사용자 지정 불용어 목록을 찾는 것은 여러번의 반복 후에만 가능합니다. 실험을 반복하여 잘 작동하고 구현 가능한 모델을 얻을 수 있는 실행 가능한 통찰력을 얻는 것이 좋습니다. 지금까지 `pycaret.nlp` 모듈을 사용하여 토픽 모델을 만들고 분석하는 방법에 대해서 알아봤습니다. 다음 섹션에서는 토픽 모델을 평가하는 방법에 대해 조금 더 자세히 알아보겠습니다.

# 7.0 토픽 모델 평가

LDA를 포함한 많은 토픽 모델은 확률론적 모델이고, 예측 밎 잠재 토픽 표현을 모두 제공합니다. 일반적으로 이러한 모델에 의해 생성된 결과는 의미있고 유용하며 감독되지 않은 훈련 과정으로 인해 이러한 가정을 평가하기 어렵다고 생각됩니다. 그럼에도 불구하고, 훈련된 모델이 객관적으로 좋은지 나쁜지를 식별하는 것뿐만 아니라 다른 모델/방법들을 비교할 수 있는 능력을 작는 것도 똑같이 중요합니다. 그러기 위해서는. 품질에 대한 객관적인 측정이 필요합니다. 전통적으로, 그리고 지금까지 많은 실용적인 응용 프로그램의 경우, 말뭉치에 대해 "올바른 것"이 학습되었는지 평가하기 위해 "eyeballing" 접근법이 사용됩니다. 이러한 정보를 최대화하고 비교할 수 있는 단일 metric으로 캡쳐하는 것이 이상적입니다. 오늘날 일반적으로 사용되는 접근법은:

- **Eye Balling 모델 :** 상위 N개의 단어 / 주제 등을 살펴봅니다
- **고유 평가 Metrics:** 모델의 해석 가능성과 의미론
- **외부 평가 Metrics:** 모델이 분류과 같은 사전 정의된 작업을 수행하는데 적합한가? (나중에 이 튜토리얼에서 대출 부실을 예측하는 classifier을 만드는 데 토픽 모델을 사용할 것입니다)
- **사람의 판단:** 토픽 모델이 문제에 대한 이해를 높여주는가?

이번 섹션에서는 `tune_model()` 기능을 사용하여 토픽 모델의 coherence(일관성) 값을 평가하는 방법에 대해 알아보겠습니다. 이어서 dataset의 `status` 열을 사용하여 기본값을 예측할 수 있는 classifier을 최적화 하기 위해 토픽 모델의 토픽 수에 대한 외부 평가를 수행할 것입니다.

__[모델 평가에 대해 자세히 알아보기](https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0)__

# 7.1 Coherence 값을 사용한 본질적 평가

**토픽 coherence(일관성)이란?** 토픽 coherence 측정은 토픽에서 점수가 높은 단어 간의 의미 유사성의 정도를 측정하여 단일 토픽에 대한 점수를 매깁니다. `tune_model()` 기능은 서로 다른 수의 주제를 가진 미리 정의된 그리드에서 반복하고 각 파라미터에 대한 모델을 만듭니다. 토픽 coherence는 다른 모델에 대해 평가되며 x축의 `# Topics` 함수로 y축의 `Coherence Score`를 갖는 그래프로 표시됩니다. 아래 예제를 참고하세요 :

In [8]:
tuned_unsupervised = tune_model(model = 'lda', multi_core = True)

IntProgress(value=0, description='Processing: ', max=25)

Output()

Best Model: Latent Dirichlet Allocation | # Topics: 16 | Coherence: 0.4741


coherence 점수가 가장 높은 모델은 본질적 평가 기준에 따른 최상의 모델입니다. 토픽 모델의 성능이 coherence 점수같이 한 숫자로 표현될 수 있다는 점은 매력적으로 들릴 수 있습니다. 하지만 단점이 없는 것은 아닙니다. coherence 점수에 대해 자세히 알아보려면 이것을 읽어보세요. __[[더 알아보기]](https://www.aclweb.org/anthology/D12-1087.pdf)__

이 튜토리얼은 coherence만을 다루었습니다. 또 다른 인기있는 방법은 perplexity(당혹감)입니다. 이전에 보지 못했던 새로운 데이터에 대해 모델이 얼마나 놀랐는지를 캡쳐하고 홀드아웃 테스트 세트의 정규화된 로그 가능도로 측정됩니다. 로그 가능도 부분에서 perplexity metric는 이전에 학습한 모형이 일부 새로운 보이지 않는 데이터에 얼마나 자주 적용되는지 측정하는 것이라고 생각될 수 있습니다. 즉 보이지 않는 데이터의 통계를 얼마나 잘 표현하거나 재현하는지 입니다. 그러나 최근 연구에 따르면 예측 가능성 (혹은 perplexity)과 인간의 판단은 종종 상관관계가 없으며, perplexity에 대한 최적화와 같이 상관관계가 없는 것도 사람이 해석 가능한 주제를 산출하지 못할 수도 있습니다. __[[참고]](https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0)__

# 7.2 Classifier을 사용한 외부 평가

사용중인 dataset은 `status`열을 사용하여 라벨링됩니다 (1 = loan default, 0 = no default). 이제 `tune_model()` 기능을 사용하여 최적의 주제 수를 결정하겠습니다. 이 경우 가장 좋은 방법은 감독 기계 학습에 대한 관심의 척도로 정의되는데, 이 경우 분류 문제이기 때문에 `Accuracy`이 됩니다. 아래 예제를 참고하십시오:

In [9]:
tuned_classification = tune_model(model = 'lda', multi_core = True, supervised_target = 'status')

IntProgress(value=0, description='Processing: ', max=25)

Output()

Best Model: Latent Dirichlet Allocation | # Topics: 2 | Accuracy : 0.867


이 예제에서 `Accuracy`는  `num_topics`가 `4`로 설정되어 있을 때 최적화됩니다. 이 경우 accuracy와 같은 지도 metric을 최적화하기 위한 주제 수는 최고의 coherence 값을 가진 모델과 다를 가능성이 매우 높습니다. 결국 어떤 것을 사용할지는 토픽 모델의 경우에 따라 완전히 달라집니다. 토픽 모델을 평가하는 것은 복잡한 주제입니다. 토픽 모델링을 하는 것이 처음이라면 이것들을 완전히 이해할 수 없을 것입니다. 더 알고싶다면, 이 __[YouTube 영상](https://www.youtube.com/watch?v=UkmIljRIG_M)__  을 보는 것을 추천합니다.

`load_experiment()`을 사용하면, 변수 `saved_experiment`의 전체 실험과 모든 중간 출력을 로드합니다. 파이썬의 목록 요소에 액세스 하는 것과 유사한 방법으로 특정 항목에 액세스할 수 있습니다. 아래 예에서는 최종 스택 ensembler에 액세스하여 `final_stack_soft_loaded` 변수에 저장합니다.

# 8.0 실험 기록

PyCaret 2.0에는 머신러닝 코드를 실행하고 나중에 결과를 시각화할 때 매개변수, 코드버전, metric, 출력 파일을 로깅하기 위한 백엔드 API 및 UI로 MLflow Tracking 구성요소가 포함되어 있습니다. 실험을 pycaret에 저장하려면 아래 예시처럼 `setup` 기능에 `log_experiment`와 `experiment_name` 파라미터를 사용하십시오.

`localhost:5000`에서 UI를 시작할 수 있습니다. 명령줄 혹은 노트북에서 MLFlow 서버를 초기화하기만 하면 됩니다. 아래 예제를 참고하십시오:

In [ ]:
# to start the MLFlow server from notebook:
!mlflow ui 

# Open localhost:5000 (아래는 UI의 예시입니다)
![title](https://i2.wp.com/pycaret.org/wp-content/uploads/2020/07/classification_mlflow_ui.png?resize=1080%2C508&ssl=1)

# 9.0 마무리 / 다음 단계?

이번 튜토리얼에서는 내재적 및 외재적 기술을 사용한 모델 평가와 같은 몇 가지 주요 개념을 다뤘습니다. `setup()`을 사용하여 사용자 지정 불용어를 제거하는 것을 포함하여 몇 가지 텍스트 전처리 단계를 수행한 후 토픽 모델을 만들고 그 결과를 지난 튜토리얼에서 만든 것고 비교했습니다. 그리고 토픽 모델을 평가하는 다양한 방법에 대해 이야기했고 LDA 모델의 coherence 값을 평가하기 위해 `tune_model()`을 사용했습니다. 또한 감독 설정에서 주제 수를 평가하기 위해 `tune_model()`을 사용했습니다 (이 경우 대출 상태를 예측하기 위해 classifier을 구축하는 데 사용했음).

다음 튜토리얼에선 `pycaret.nlp`를 `pycaret.classification`과 함께 사용하고 pycaret의 감독된 모듈과 감독되지 않은 모듈을 함께 사용하는 데 초점을 맞출 것입니다.

다음 튜토리얼에서 봅시다. 링크 : __[자연어 처리 튜토리얼 (NLP103) - Level Expert](https://github.com/pycaret/pycaret/blob/master/tutorials/Natural%20Language%20Processing%20Tutorial%20Level%20Expert%20-%20NLP103.ipynb)__